In [6]:
print("Beginning imports")
import pygame
from shapely.geometry import *
from shapely.affinity import *
from math import *
from shapely.ops import nearest_points
from shapely.geometry.polygon import LinearRing
import numpy as np
import keras
import sys
from copy import deepcopy
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint, EarlyStopping
import os
import time
import datetime


Beginning imports


In [7]:
directionArray = [["w"],["a"],["d"],["w","a"],["w","d"],["s"],["s","a"],["s","d"]]
numOfOutputs = 5 # first 5 from directionArray as choices for the NN
numOfInputs = 24 # look every 360/24 = 15 degrees
death_penalty = -1 #score on death
LEARNING_RATE = 0.005 #Adam LR value
visualization_scale_multiplier = 20 #Used control pygame windows size
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
MAGENTA = (255, 0, 255)
CYAN = (0, 255, 255)
BROWN = (210, 105, 30)
np.random.seed(123)
verbose = 0 #increasing to 1 will print out a lot of diagnostic information

print("Initializing track")
#trackballs and rings are the track walls
trackball = [(0,0),(0, 4), (0, 8), (2, 12), (2, 16), (0, 20), (-2, 24), (-6, 28), (-2,32), (0, 32), (4, 32), (8, 32), (12, 32), (16,32), (20, 32), (20, 28), (20,24),(20,20),(20,16),(21, 17), (21,24),(21,28),(21,32),(24, 32), (28,32), (32,32), (36,32), (36,28),(36, 24),(36,20),(36, 16), (36,12),(36, 8), (36, 4), (36, 0), (32, 0), (28, 0), (24, 0), (20, 0), (16, 0), (12, 0), (8, 0), (4, 0), (0,0)]
ring = LinearRing(trackball)
trackball2 = [(4, 4), (4, 8), (8, 12), (8, 16), (4, 20), (4, 24), (0, 28), (4, 28), (8, 28), (12, 24), (12, 20), (12, 16), (16, 12), (12, 8), (16, 8), (20, 8), (24, 8), (28, 8), (28, 12), (28, 16), (28, 20), (28, 24), (28, 28), (32,28), (32, 4), (4, 4)]
ring2 = LinearRing(trackball2)

#Getting dimensions of the track and diagonal length of the track
X_range = (min([min([a_tuple[0] for a_tuple in trackball]),min([a_tuple[0] for a_tuple in trackball2])]), max([max([a_tuple[0] for a_tuple in trackball]),max([a_tuple[0] for a_tuple in trackball2])]))
y_range = (min([min([a_tuple[1] for a_tuple in trackball]),min([a_tuple[1] for a_tuple in trackball2])]), max([max([a_tuple[1] for a_tuple in trackball]),max([a_tuple[1] for a_tuple in trackball2])]))
diag_distance_of_track = ((y_range[0] - y_range[1])**2 + (X_range[0] - X_range[1])**2)**(1/2)

#Creating trackballs that is easily digestable by pygame
visualization_trackball = [((xtemp - X_range[0]) * visualization_scale_multiplier, (ytemp - y_range[0]) * visualization_scale_multiplier) for xtemp, ytemp in trackball]
visualization_trackball2 = [((xtemp - X_range[0]) * visualization_scale_multiplier, (ytemp - y_range[0]) * visualization_scale_multiplier) for xtemp, ytemp in trackball2]

masterGatesToDo = [] #Holds all successgates in a digestable way
stochastic_action_preprocessed_range = range(0,numOfOutputs) #Holds the number of moves available to the agent

vision_array_preprocessed = [] #Holds the degrees to look at (0, 15, 30 ... 330, 345)
for i in range(0,numOfInputs):
    vision_array_preprocessed.append(i*(360/(numOfInputs)))

#Organizing Success gates begins (gas pickups)
successGates = []
for i in trackball[:-1]:
    successGates.append(LineString([Point(i), Point(nearest_points(Point(i),ring2)[1])]))
    
successGates.append(successGates.pop(0))
successGates.append(successGates.pop(0))

#Manually reorganizing gates that were generated in successGates
tempGates = []
for i in range(0,12):
    tempGates.append(successGates[i])
for i in range(14,20):
    tempGates.append(successGates[i])
tempGates.append(successGates[13])
tempGates.append(successGates[12])
for i in range(20,len(successGates)):
    tempGates.append(successGates[i])
    
tempGates.pop(16)
successGates = tempGates
successGatesLength = len(successGates)

for i in range(0,successGatesLength):
    masterGatesToDo.append(i)

#Organizing Success gates Ends


scaler = MinMaxScaler(feature_range=(0, 1)) #Used for scaling Q values to 0 - 1

print("Initializing car")
class car:
    def __init__(self,x,y,discount=0.99,step_size=0.99, number_of_hidden_units = 12):
        self.xy = Point(x,y)
        x2 = x
        y2 = y + diag_distance_of_track
        self.direction = LineString([(x,y),(x2,y2)])
        self.angle = 90
        self.prev_vision_array = []
        self.gatesToDo = deepcopy(masterGatesToDo)
        self.vision_array = []
        self.np_vision_array = np.array([])
        self.gate_vision_array = []
        self.np_gate_vision_array = np.array([])
        self.vision()
        self.score = 0
        self.frameCount = 0
        self.framesSinceProgress = 0
        self.model = self.create_model(number_of_hidden_units)        
        self.longTermMemory={}
        self.longTermMemoryVisionArray={}
        self.prevAction=0
        self.prevState = [str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle])]
        self.state = [str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle])]
        self.discount = discount
        self.step_size = step_size
        self.BestPerformance = 0
        
    def drive(self,inputKeys):
        #rotates and moves the car based off of inputKeys
        distance = 0
        angle = 0
        for i in inputKeys:
            if i == "w":
                distance+= 1
                
            elif i == "a":
                angle-=15
                
            elif i == "s":
                distance-= 1
                
            elif i == "d":
                angle+=15

        self.angle = (angle + self.angle) % 360
        self.xy = Point(round(self.xy.coords[0][0] + cos(radians(self.angle)) * distance,2),
                        round(self.xy.coords[0][1] + sin(radians(self.angle)) * distance,2))
        self.direction = rotate(LineString([self.xy,(Point(self.direction.coords[1][0] + cos(radians(self.angle)) * distance,
        self.direction.coords[1][1] + sin(radians(self.angle)) * distance))]),angle,origin=self.xy)

    def vision(self):
        #Looks out from the carevery x number of degrees and gets distances between the car and the 
        #wall at the intersection of the vision beam
        #It also does this for distances of the success gates
        #0.5 is the radius of the car, so we look at the distances from the edge of the car
        #The np_vision_array are normalized by the diagonal length of the track

        self.vision_array = []
        self.gate_vision_array = []
        for i in vision_array_preprocessed:
            temp = rotate(self.direction,i,origin=self.xy)
            dist = np.ceil(diag_distance_of_track)
            if(temp.intersects(ring)):
                if (type(temp.intersection(ring))) == Point:
                    if self.xy.distance(temp.intersection(ring)) < dist:
                        dist = self.xy.distance(temp.intersection(ring))

                elif (type(temp.intersection(ring))) == LineString:
                    if self.xy.distance(nearest_points(self.xy,temp.intersection(ring))[1]) < dist:
                        dist = self.xy.distance(nearest_points(self.xy,temp.intersection(ring))[1])

                elif type(temp.intersection(ring)) == MultiPoint:
                    for tempPoint in temp.intersection(ring):
                        if self.xy.distance(tempPoint) < dist:
                            dist = self.xy.distance(tempPoint)
                            
                elif type(temp.intersection(ring)) == GeometryCollection:
                    for tempPoint2 in temp.intersection(ring):
                        if (type(tempPoint2)) == Point:
                            if self.xy.distance(tempPoint2) < dist:
                                dist = self.xy.distance(tempPoint2)
                               
                        elif (type(tempPoint2)) == LineString:
                            if self.xy.distance(nearest_points(self.xy,tempPoint2)[1]) < dist:
                                dist = self.xy.distance(nearest_points(self.xy,tempPoint2)[1])
                              
                        else:
                            print(type(temp.intersection(ring)))
                            raise NotImplementedError   
                            
                else:
                    print(type(temp.intersectrion(ring)))
                    raise NotImplementedError
                
            if(temp.intersects(ring2)):
                if (type(temp.intersection(ring2))) == Point:
                    if self.xy.distance(temp.intersection(ring2)) < dist:
                        dist = self.xy.distance(temp.intersection(ring2))
                        
                elif (type(temp.intersection(ring2))) == LineString:
                    if self.xy.distance(nearest_points(self.xy,temp.intersection(ring2))[1]) < dist:
                        dist = self.xy.distance(nearest_points(self.xy,temp.intersection(ring2))[1])
                       
                elif type(temp.intersection(ring2)) == MultiPoint:
                    for tempPoint in temp.intersection(ring2):
                        if self.xy.distance(tempPoint) < dist:
                            dist = self.xy.distance(tempPoint)
                            
                elif type(temp.intersection(ring2)) == GeometryCollection:
                    for tempPoint2 in temp.intersection(ring2):
                        if (type(tempPoint2)) == Point:
                            if self.xy.distance(tempPoint2) < dist:
                                dist = self.xy.distance(tempPoint2)
                               
                        elif (type(tempPoint2)) == LineString:
                            if self.xy.distance(nearest_points(self.xy,tempPoint2)[1]) < dist:
                                dist = self.xy.distance(nearest_points(self.xy,tempPoint2)[1])
                               
                        else:
                            print(type(temp.intersection(ring2)))
                            raise NotImplementedError
                            
                else:
                    print(type(temp.intersection(ring2)))
                    raise NotImplementedError
                    
            if (dist == 99999999):
                dist = 0.5
            
            self.vision_array.append(dist - 0.5)
            
            for i in self.gatesToDo[0:min(10,len(self.gatesToDo))]:
                if(temp.intersects(successGates[i])):
                    if (type(temp.intersection(successGates[i]))) == Point:
                        if self.xy.distance(temp.intersection(successGates[i])) < dist:
                            dist = self.xy.distance(temp.intersection(successGates[i]))
                        
                    elif (type(temp.intersection(successGates[i]))) == LineString:
                        if self.xy.distance(nearest_points(self.xy,temp.intersection(successGates[i]))[1]) < dist:
                            dist = self.xy.distance(nearest_points(self.xy,temp.intersection(successGates[i]))[1])

                    elif type(temp.intersection(successGates[i])) == MultiPoint:
                        for tempPoint in temp.intersection(successGates[i]):
                            if self.xy.distance(tempPoint) < dist:
                                dist = self.xy.distance(tempPoint)

                    elif type(temp.intersection(successGates[i])) == GeometryCollection:
                        for tempPoint2 in temp.intersection(successGates[i]):
                            if (type(tempPoint2)) == Point:
                                if self.xy.distance(tempPoint2) < dist:
                                    dist = self.xy.distance(tempPoint2)

                            elif (type(tempPoint2)) == LineString:
                                if self.xy.distance(nearest_points(self.xy,tempPoint2)[1]) < dist:
                                    dist = self.xy.distance(nearest_points(self.xy,tempPoint2)[1])

                            else:
                                print(type(temp.intersection(successGates[i])))
                                raise NotImplemented

                    else:
                        print(type(temp.intersection(successGates[i])))
                        raise NotImplemented
                   
            if dist - 0.5 < self.vision_array[-1]:
               if verbose >= 1:
                   print("I see a gate!")
            
            else:
                dist = 0.5
            
            self.gate_vision_array.append(dist - 0.5)                   
        self.np_vision_array = np.array([self.vision_array, self.gate_vision_array]).flatten('F').reshape(1, numOfInputs * 2) / diag_distance_of_track

    def gateCheck(self):
        #Checks if the car is touching a successgate
        gateToCheckBall = []
        gatesToDoBall = []
        for i in self.gatesToDo[0:min(10,len(self.gatesToDo))]:
            gateToCheckBall.append(successGates[i])

        for gateToCheck in gateToCheckBall:
            if (self.xy.distance(gateToCheck) <= 0.5001):
                self.framesSinceProgress = 0 
                self.score+=1
                if (len(self.gatesToDo) == 1): #If all gates have been touched
                    self.gatesToDo = []
                    self.score+= 50
                    self.gatesToDo = deepcopy(masterGatesToDo)
                    global notDone
                    notDone = False
                
                self.gatesToDo.pop(self.gatesToDo.index(successGates.index(gateToCheck)))    

    def deathCheck(self):
        #Checks if the car has died
        if self.framesSinceProgress == 25:
            if verbose:
                print("Death by: Timed out since progress")
            return True

        if self.xy.distance(ring) < 0.5 or self.xy.distance(ring2) < 0.5:
            if verbose:
                print("Death by: Crashed")
            return True
        
        if self.frameCount == 100000:
            if verbose:
                print("Death by: surviving for 100000 frames")
            return True
        
        return False
        
    
    def frame(self,inputArray):
        #Moves the car forward one frame
        self.drive(inputArray)
        self.vision()
        if verbose:
            print(self.np_vision_array)
            
        self.frameCount+=1
        self.framesSinceProgress+=1
    
    def self_drive(self):
        #self drive the car using the neural network to control what inputs to make
        while True:
            self.prevScore = self.score
            self.prevState =str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle]) 
            self.state = self.prevState
            self.prevAction = self.memoryFetch()
            if verbose:
                print("State: " +  str(self.prevState) + "\nAction: " + str(self.prevAction))
            self.frame(directionArray[self.prevAction])
            self.state =str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle]) 
            if self.state not in self.longTermMemory:
                action = np.random.choice(range(0,numOfOutputs))
                self.longTermMemoryVisionArray[self.state] = self.np_vision_array
                self.longTermMemory[self.state] = np.zeros(5)
                
            self.score-=0.01 #Movement cost
            self.gateCheck()
            if not self.deathCheck():
                self.memoryCommit()
            else:
                #print("after:",self.longTermMemory)
                break

        if self.score > self.BestPerformance:
            print(f"Score {self.score:.2f} Passed success gates:", set(masterGatesToDo)-set(self.gatesToDo))
            self.BestPerformance = self.score
            global gatesPerc
            if self.score > successGatesLength - 1:
                gatesPerc = 1
            else:
                gatesPerc = ((len(masterGatesToDo) - len(self.gatesToDo))/len(masterGatesToDo))

        self.score+= death_penalty
        self.memoryCommit()
        return self.score
            
    def reset_and_self_drive(self):
        #Refresh the cars initial state and self drive
        self.xy = deepcopy(init_xy)
        self.direction = deepcopy(init_direction)
        self.angle = deepcopy(init_angle)
        self.vision_array = deepcopy(init_vision_array)
        self.np_vision_array = deepcopy(init_np_vision_array)
        self.score = deepcopy(init_score)
        self.frameCount = deepcopy(init_frameCount)
        self.gatesToDo = deepcopy(masterGatesToDo)
        self.framesSinceProgress = deepcopy(init_framesSinceProgress)
        self.prevAction=""
        self.prevState=0
        return self.self_drive()
    
    def memoryFetch(self):
        #Get a selection from the neural network and initialize the long term memory with a new state if its unique
        if ( self.prevState in self.longTermMemory ):
            if verbose:
                print("Model prediction:",self.model.predict(x=[self.np_vision_array,np.array([1] * numOfOutputs).reshape(1,numOfOutputs)]))

            action = np.argmax(self.model.predict(x=[self.np_vision_array, np.array([1] * numOfOutputs).reshape(1,numOfOutputs)]))
            randAction = np.random.choice(stochastic_action_preprocessed_range, p = stochastic_action_preprocessed_probability_array)
            if verbose and randAction != 0:
                print("Random step")
            action = (action + randAction) % numOfOutputs

        else:
            action = np.random.choice(range(0,numOfOutputs))
            self.longTermMemory[self.prevState] = np.zeros(numOfOutputs)
            self.longTermMemoryVisionArray[self.prevState] = self.np_vision_array
               
        return( action )
    
    def memoryCommit(self):
        #Apply the Q learning algorithm to the long term memory using the SARS paradigm
        scoreShift = self.score - self.prevScore
        if  verbose:
            print("before value self.longTermMemory[self.prevState][self.prevAction]:",self.longTermMemory[self.prevState][self.prevAction])
            print("scoreShift: ", scoreShift)
            
        self.longTermMemory[self.prevState][self.prevAction] = self.longTermMemory[self.prevState][self.prevAction] + self.discount * (scoreShift + self.step_size * (np.max(self.longTermMemory[self.state])) - self.longTermMemory[self.prevState][self.prevAction])
        
        if  verbose:
            print("New value self.longTermMemory[self.prevState][self.prevAction]:",self.longTermMemory[self.prevState][self.prevAction])
        
    def create_model(self, number_of_hidden_units):
        #Neural network model initialization
        n_actions = numOfOutputs
        obs_shape = ((numOfInputs * 2),)
        observations_input = keras.layers.Input(obs_shape, name='observations_input')
        action_mask = keras.layers.Input((n_actions,), name='action_mask')
        hidden = keras.layers.Dense(number_of_hidden_units, activation='relu')(observations_input)
        hidden_2 = keras.layers.Dense(number_of_hidden_units, activation='relu')(hidden)
        output = keras.layers.Dense(n_actions)(hidden_2)
        filtered_output = keras.layers.multiply([output, action_mask])
        model = keras.models.Model([observations_input, action_mask], filtered_output)
        optimizer = keras.optimizers.Adam(lr=LEARNING_RATE, clipnorm=1.0)
        model.compile(optimizer, loss='mean_squared_error')
        return model
    
    def self_drive_and_visualize(self):
        #Used to visualize the car going around the track
        pygame.init()
        pygame.display.set_caption("DRL CAR")
        car_img = pygame.image.load("final_project/images/circle_with_arrow.png")
        screen = pygame.display.set_mode(((X_range[1] - X_range[0]) * visualization_scale_multiplier, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 200))
        car_img_original = pygame.transform.scale(car_img,(visualization_scale_multiplier,visualization_scale_multiplier))
        while self.score < 90:
            screen.fill(WHITE)
            pygame.draw.polygon(screen, BLACK, visualization_trackball, visualization_scale_multiplier)
            pygame.draw.polygon(screen, BLACK, visualization_trackball2, visualization_scale_multiplier)
            for i in self.gatesToDo:
                pygame.draw.line(screen, MAGENTA, 
                                ((successGates[i].coords[:][0][0] - X_range[0]) * visualization_scale_multiplier,
                                 (successGates[i].coords[:][0][1] - y_range[0]) * visualization_scale_multiplier),
                                 ((successGates[i].coords[:][1][0] - X_range[0]) * visualization_scale_multiplier,
                                 (successGates[i].coords[:][1][1] - y_range[0]) * visualization_scale_multiplier),
                                 visualization_scale_multiplier)
            
            
            for i,temp_angle in enumerate(vision_array_preprocessed):
                temp_angle = (temp_angle + self.angle) % 360
                pygame.draw.line(screen, CYAN,
                ((round(self.xy.coords[0][0] + cos(radians(temp_angle)) * (self.np_vision_array[0][i * 2] ) * diag_distance_of_track, 2) - X_range[0]) * visualization_scale_multiplier, 
                 (round(self.xy.coords[0][1] + sin(radians(temp_angle)) * (self.np_vision_array[0][i * 2] ) * diag_distance_of_track, 2) - y_range[0]) * visualization_scale_multiplier),
                ((self.xy.coords[0][0] - X_range[0]) * visualization_scale_multiplier,
                 (self.xy.coords[0][1] - y_range[0]) * visualization_scale_multiplier),
                 int(visualization_scale_multiplier/4))
                
                pygame.draw.line(screen, BROWN,
                ((round(self.xy.coords[0][0] + cos(radians(temp_angle)) * (self.np_vision_array[0][i * 2 + 1] ) * diag_distance_of_track, 2) - X_range[0]) * visualization_scale_multiplier, 
                 (round(self.xy.coords[0][1] + sin(radians(temp_angle)) * (self.np_vision_array[0][i * 2 + 1] ) * diag_distance_of_track, 2) - y_range[0]) * visualization_scale_multiplier),
                ((self.xy.coords[0][0] - X_range[0]) * visualization_scale_multiplier,
                 (self.xy.coords[0][1] - y_range[0]) * visualization_scale_multiplier),
                 int(visualization_scale_multiplier/4))
                
            font = pygame.font.Font('freesansbold.ttf', 24) 
            nn_title = font.render("Neural Network Predictions", True, BLACK, WHITE)
            best_value_marker = "^"
            nn_values = self.model.predict(x=[self.np_vision_array,np.array([1] * numOfOutputs).reshape(1,numOfOutputs)])[0]
            action_labels = font.render("".join(("".join(x)).ljust(6, " ") for x in directionArray[:numOfOutputs]), True, BLACK, WHITE)
            action_values = font.render("".join(str(round(x,2)).ljust(6, " ") for x in nn_values), True, BLACK, WHITE)
            action_best_value = font.render("".join(best_value_marker), True, BLACK, WHITE)
            distance_from_wall = font.render("Distance From wall: " + str(round(min(self.xy.distance(ring), self.xy.distance(ring2)) - 0.5, 2)), True, BLACK, WHITE)
            car_score = font.render("Score: " + str(round(self.score, 2)), True, BLACK, WHITE)
            gas_remaining = font.render("Gas remaining: " + str(25 - self.framesSinceProgress), True, BLACK, WHITE)
            steps_taken_total = font.render("Steps taken: " + str(self.frameCount), True, BLACK, WHITE)
            nn_title_rect = nn_title.get_rect()
            action_labels_rect = action_labels.get_rect()
            action_values_rect = action_values.get_rect()  
            action_best_rect = action_best_value.get_rect()
            distance_from_wall_rect = nn_title.get_rect()
            car_score_rect = car_score.get_rect()
            gas_remaining_rect = gas_remaining.get_rect()
            steps_taken_total_rect = steps_taken_total.get_rect()
            nn_title_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 35)
            action_labels_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 75)
            action_values_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 115)
            action_best_rect.center = ((X_range[1] - X_range[0] - 25 + np.argmax(nn_values) * 12) * visualization_scale_multiplier // 4, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 155)
            distance_from_wall_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4 * 3, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 35)
            car_score_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4 * 3, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 75)
            gas_remaining_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4 * 3, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 115)
            steps_taken_total_rect.center = ((X_range[1] - X_range[0]) * visualization_scale_multiplier // 4 * 3, (y_range[1] - y_range[0]) * visualization_scale_multiplier + 155)
            car_img = pygame.transform.flip(pygame.transform.rotate(car_img_original, self.angle), True, False)
            screen.blit(car_img,((self.xy.coords[:][0][0] - X_range[0]) * visualization_scale_multiplier - int(visualization_scale_multiplier / 2), (self.xy.coords[:][0][1] - y_range[0]) * visualization_scale_multiplier - int(visualization_scale_multiplier / 2)))
            screen.blit(nn_title, nn_title_rect)
            screen.blit(action_labels, action_labels_rect)
            screen.blit(action_values, action_values_rect)
            screen.blit(action_best_value, action_best_rect)
            screen.blit(distance_from_wall, distance_from_wall_rect)
            screen.blit(car_score, car_score_rect)
            screen.blit(gas_remaining, gas_remaining_rect)
            screen.blit(steps_taken_total, steps_taken_total_rect)
            pygame.display.update()

            self.prevScore = self.score
            self.prevState = str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle]) 
            self.state = self.prevState
            self.prevAction = self.memoryFetch()
            if verbose:
                print("State: " +  str(self.prevState) + "\nAction: " + str(self.prevAction))
                
            self.frame(directionArray[self.prevAction])
            self.state = str([self.xy.coords[0][0],self.xy.coords[0][1], self.angle]) 
            if self.state not in self.longTermMemory:
                action = np.random.choice(range(0,numOfOutputs))
                self.longTermMemoryVisionArray[self.state] = self.np_vision_array
                self.longTermMemory[self.state] = np.zeros(5)

            self.score-=0.01 #Movement cost
            self.gateCheck()
            time.sleep(0.25)

            if not self.deathCheck():
                self.memoryCommit()
                
            else:
                break

        if self.score > self.BestPerformance:
            print(f"Score {self.score:.2f} Passed success gates:", set(masterGatesToDo)-set(self.gatesToDo))
            self.BestPerformance = self.score
            global gatesPerc
            if self.score > successGatesLength - 1:
                gatesPerc = 1
            else:
                gatesPerc = ((len(masterGatesToDo) - len(self.gatesToDo))/len(masterGatesToDo))

        self.score+= death_penalty
        self.memoryCommit()
        return self.score

def fit_batch2(bench_mark, callBackser = None):
    #Trains the neural network on the long term memory
    count = 0
    if callBackser == None: 
        numOfEpochs = 5
        val_split = 0
    else:
        numOfEpochs = 9999999
        if callBackser == "val_loss" or callBackser == "val_acc":
            callBackser = EarlyStopping(monitor = callBackser, patience = 10, verbose = verbose, restore_best_weights = True)
            val_split = 0.33
            
        elif callBackser == "checkpointer_earlystoper_car_score":
            callBackser = [EarlyStoppingAtBestCarScore(patience = 15, best_car_score = bench_mark, init_weights = googleCar.model.get_weights())]
            val_split = 0
            
        else:
            raise NotImplementedError
            
    
    batch_vision_array = [np.array([]),np.array([]),np.array([])]
    batch_vision_array_ball = []

    for key in (googleCar.longTermMemory):
        if googleCar.longTermMemory[key].any != 0: 
            count+=1
            temp=key.split(",")
            oheTemp = np.array([])
            for tempVal in googleCar.longTermMemory[key]:
                if tempVal == 0:
                    oheTemp = np.append(oheTemp, 0)
                else:
                    oheTemp = np.append(oheTemp, 1)

            batch_vision_array_ball.append([googleCar.longTermMemoryVisionArray[key], googleCar.longTermMemory[key], oheTemp])
    
    for trio in batch_vision_array_ball:
        batch_vision_array[0] = np.append(batch_vision_array[0],trio[0]) #Vision values
        batch_vision_array[1] = np.append(batch_vision_array[1],trio[1]) #score values
        batch_vision_array[2] = np.append(batch_vision_array[2],trio[2]) #OHE
    
    batch_vision_array[0] = batch_vision_array[0].reshape(int(len(batch_vision_array[0])/(numOfInputs*2)),(numOfInputs*2))
    batch_vision_array[1] = scaler.fit_transform(batch_vision_array[1].reshape(int(len(batch_vision_array[1])/numOfOutputs),numOfOutputs))
    batch_vision_array[2] = batch_vision_array[2].reshape(int(len(batch_vision_array[2])/numOfOutputs),numOfOutputs)
    
    if False: #counter % 200 == 0:
        verbose = 1
    else:
        verbose = 0
    
    
    googleCar.model.fit([batch_vision_array[0], batch_vision_array[2]], batch_vision_array[1], batch_size=64, epochs=numOfEpochs, verbose = verbose, callbacks = callBackser, validation_split = val_split) 
        
class EarlyStoppingAtBestCarScore(keras.callbacks.Callback):
    #This is a custom callback that tests the performance of the model and stores the best
    #performing model based off the cars score
    def __init__(self, patience=0, best_car_score=-999, init_weights=None):
        super(EarlyStoppingAtBestCarScore, self).__init__()
        self.patience = patience
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = init_weights
        self.best = best_car_score

    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minimum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        #self.best = -(np.Inf)

    def on_epoch_end(self, epoch, logs=None):
        current = googleCar.reset_and_self_drive()
        #logs.get("loss")
        if np.less(self.best, current):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                if verbose >= 1:
                    print("Restoring model weights from the end of the best epoch.")
                self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            if verbose >= 1:
                print("Epoch %05d: early stopping" % (self.stopped_epoch + 1))
print("Done")


Initializing track
Initializing car
Done


In [8]:
#Initialize values for reseting the cars position at the beginning of each episode
googleCar = car(2,6)
print("Initializing init values")
init_xy = deepcopy(googleCar.xy)
init_direction = deepcopy(googleCar.direction)
init_angle = deepcopy(googleCar.angle)
init_vision_array = deepcopy(googleCar.vision_array)
init_np_vision_array = deepcopy(googleCar.np_vision_array)
init_score = deepcopy(googleCar.score)
init_frameCount = deepcopy(googleCar.frameCount)
init_framesSinceProgress = deepcopy(googleCar.framesSinceProgress)
snapExploitation = 1

Initializing init values


In [10]:
#Comment out if you don't want to visualize
#Choose a model to visualize
googleCar.model = keras.models.load_model('2_1_4269.708544492722_2567')

#Make the decision making non stochastic
stochastic_action_preprocessed_probability_array = [snapExploitation]
for i in range(0, numOfOutputs-1):
    stochastic_action_preprocessed_probability_array.append((snapExploitation-1)/(numOfOutputs-1))


In [ ]:
#Comment out if you don't want to visualize
#Visualize the selected model in pygame
googleCar.self_drive_and_visualize()

In [6]:
#uncomment to test configurations
# callbacksersList = ["checkpointer_earlystoper_car_score", "val_loss", "val_acc"] #Done
# numberOfHiddenUnitsList = [24, 12, 48] #Done
# numOfInputsList = [24, 12, 48] #Done
# numOfOutputsList = [5, 8] #Done
# death_penaltyList = [-1, -100, 0] #Done
# discountList = [0.99, 0.25, 1] #Done
# stepList = [0.99, 0.25, 1] #Done
# learning_rateList = [0.005, 0.001, 0.01] #Done
# loopLengthList = [150, 1, 25, 1000] #Done
# stochasticismRoofFloorList = [(0.40,0.90), (0.20,0.99), (0.90,0.90), (0.001,0.50), (1,1)]
# anyOrAllList = ["any_without_zeroes", "all", "any_with_zeroes"]

In [7]:
#uncomment to test configurations

# loopLength = 150
# verbose = 0
# start_time = time.time()
# print("Start epoch time: ", start_time)
# now = datetime.datetime.now()
# print ("Start date and time : ")
# print (now.strftime("%Y-%m-%d %H:%M:%S"))


# def collect_attempts(num_of_attempts, max_time, model_name):
#     results = []
#     for attempt_num in range(num_of_attempts):
#         results.append(attempt(attempt_num, max_time, model_name))
    
#     return results

# def attempt(attempt_number, max_time, model_name): #max_time in seconds
#     global gatesPerc
#     global exploitationBase
#     global exploitation
#     global exploration
#     global stochastic_action_preprocessed_probability_array
#     global googleCar

#     gatesPerc = 0
#     exploitationBase = stochasticismFloorTemp
#     exploitation = min(stochasticismFloorTemp + gatesPerc, stochasticismRoofTemp)
#     exploration = 1 - exploitation
#     stochastic_action_preprocessed_probability_array = [exploitation]
#     for i in range(0, numOfOutputs-1):
#         stochastic_action_preprocessed_probability_array.append(exploration/(numOfOutputs-1))
    
#     googleCar = car(x = 2, 
#                     y = 6, 
#                     discount = discountTemp, 
#                     step_size = stepTemp,
#                     number_of_hidden_units = numOfHiddenUnitsTemp)

#     attempt_start_time = time.time()
#     print(f"Starting attempt {attempt_number} time: ", start_time)
#     print(f'Start date and time : {datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')
#     counter = 0
#     last_print = 0
#     while time.time() - attempt_start_time < max_time: 
#         counter+=1
#         for i in range(0,loopLength):
#             googleCar.reset_and_self_drive()

#         stochastic_action_preprocessed_probability_array = [snapExploitation]
#         for i in range(0, numOfOutputs-1):
#             stochastic_action_preprocessed_probability_array.append((snapExploitation-1)/(numOfOutputs-1))

#         exploitation_bench_mark = googleCar.reset_and_self_drive()
        
#         if counter * loopLength - 500 > last_print:
#             print(f"exploitation {exploitation_bench_mark:.2f}  Number of cars driven:", str(counter*(loopLength+1)))
#             last_print = counter * loopLength

#         if exploitation_bench_mark - death_penalty > successGatesLength:
#             print("SUCCESS!!!!")
#             print("time taken:", time.time() - attempt_start_time)
#             googleCar.model.save(model_name + "_" + str(attempt_number) + "_" + str(time.time() - attempt_start_time) + "_" + str(counter*(loopLength+1)))
#             now = datetime.datetime.now()
#             print ("End date and time : ")
#             print (now.strftime("%Y-%m-%d %H:%M:%S"))
#             break
        
#         if anyOrAllTemp == "any_with_zeroes":
#             fit_batch2_any_with_zeroes(bench_mark = exploitation_bench_mark, callBackser = callbacksersTemp)
            
#         elif anyOrAllTemp == "any_without_zeroes":
#             fit_batch2_any_without_zeroes(bench_mark = exploitation_bench_mark, callBackser = callbacksersTemp)
            
#         elif anyOrAllTemp == "all":
#             fit_batch2_all(bench_mark = exploitation_bench_mark, callBackser = callbacksersTemp)
            
#         else:
#             print(f"anyOrAllTemp: {anyOrAllTemp}")
#             raise NotImplementedError

#         exploitationBase = min(stochasticismFloorTemp + 0.0001, stochasticismRoofTemp)
#         exploitation = min(stochasticismFloorTemp + gatesPerc, stochasticismRoofTemp)
#         exploration = 1 - exploitation
#         stochastic_action_preprocessed_probability_array = [exploitation]
#         for i in range(0, numOfOutputs-1):
#             stochastic_action_preprocessed_probability_array.append(exploration/(numOfOutputs-1))
    
#     if time.time() - attempt_start_time < max_time:
#         return [time.time() - start_time, counter*(loopLength+1)]
    
#     else:
#         print("TIMED OUT!")
#         return [max_time, counter*(loopLength+1)]

In [8]:
#uncomment to test configurations

# def fit_batch2_any_with_zeroes(bench_mark, callBackser = None):
#     count = 0
#     # callbacksersList = ["val_loss", "val_acc" "checkpointer_earlystoper_car_score", None]
        
#     if False: #counter % 200 == 0:
#         verbose = 1
#     else:
#         verbose = 0
    
    
#     if callBackser == None:
#         numOfEpochs = 5
#         val_split = 0
#     else:
#         numOfEpochs = 9999999
#         if callBackser == "val_loss" or callBackser == "val_acc":
#             #checkpointer_val_loss = ModelCheckpoint("temp.hdf5", monitor="val_loss", mode="auto", save_best_only = True, verbose = verbose)
#             callBackser = [EarlyStopping(monitor = callBackser, patience = 10, verbose = verbose, restore_best_weights = True)]
#             val_split = 0.33
#         elif callBackser == "checkpointer_earlystoper_car_score":
#             callBackser = [EarlyStoppingAtBestCarScore(patience = 15, best_car_score = bench_mark, init_weights = googleCar.model.get_weights())]
#             val_split = 0
#         else:
#             raise NotImplementedError
            
#     batch_vision_array = [np.array([]),np.array([]),np.array([])]
#     batch_vision_array_ball = []

#     for key in (googleCar.longTermMemory):
#         if googleCar.longTermMemory[key].any != 0: 
#             count+=1
#             temp=key.split(",")
#             #oheTemp = np.array([1,1,1,1,1])
# #             for tempVal in googleCar.longTermMemory[key]:
# #                 if tempVal == 0:
# #                     oheTemp = np.append(oheTemp, 0)
# #                 else:
# #                     oheTemp = np.append(oheTemp, 1)

#             batch_vision_array_ball.append([googleCar.longTermMemoryVisionArray[key], googleCar.longTermMemory[key], np.array([1,1,1,1,1])])
    
#     for trio in batch_vision_array_ball:
#         batch_vision_array[0] = np.append(batch_vision_array[0],trio[0]) #Vision values
#         batch_vision_array[1] = np.append(batch_vision_array[1],trio[1]) #score values
#         batch_vision_array[2] = np.append(batch_vision_array[2],trio[2]) #OHE
    
#     batch_vision_array[0] = batch_vision_array[0].reshape(int(len(batch_vision_array[0])/(numOfInputs*2)),(numOfInputs*2))
#     batch_vision_array[1] = scaler.fit_transform(batch_vision_array[1].reshape(int(len(batch_vision_array[1])/numOfOutputs),numOfOutputs))
#     batch_vision_array[2] = batch_vision_array[2].reshape(int(len(batch_vision_array[2])/numOfOutputs),numOfOutputs)
    
    
#     googleCar.model.fit([batch_vision_array[0], batch_vision_array[2]], batch_vision_array[1], batch_size=64, epochs=numOfEpochs, verbose = verbose, callbacks = callBackser, validation_split = val_split) 

# def fit_batch2_any_without_zeroes(bench_mark, callBackser = None):
#     count = 0
#     # callbacksersList = ["val_loss", "val_acc" "checkpointer_earlystoper_car_score", None]
        
#     if False: #counter % 200 == 0:
#         verbose = 1
#     else:
#         verbose = 0
    
#     if callBackser == None:
#         numOfEpochs = 5
#         val_split = 0
#     else:
#         numOfEpochs = 9999999
#         if callBackser == "val_loss" or callBackser == "val_acc":
#             #checkpointer_val_loss = ModelCheckpoint("temp.hdf5", monitor="val_loss", mode="auto", save_best_only = True, verbose = verbose)
#             callBackser = [EarlyStopping(monitor = callBackser, patience = 10, verbose = verbose, restore_best_weights = True)]
#             val_split = 0.33
#         elif callBackser == "checkpointer_earlystoper_car_score":
#             callBackser = [EarlyStoppingAtBestCarScore(patience = 15, best_car_score = bench_mark, init_weights = googleCar.model.get_weights())]
#             val_split = 0
#         else:
#             raise NotImplementedError
            
#     batch_vision_array = [np.array([]),np.array([]),np.array([])]
#     batch_vision_array_ball = []

#     for key in (googleCar.longTermMemory):
#         if googleCar.longTermMemory[key].any != 0: 
#             count+=1
#             temp=key.split(",")
#             oheTemp = np.array([])
#             for tempVal in googleCar.longTermMemory[key]:
#                 if tempVal == 0:
#                     oheTemp = np.append(oheTemp, 0)
#                 else:
#                     oheTemp = np.append(oheTemp, 1)

#             batch_vision_array_ball.append([googleCar.longTermMemoryVisionArray[key], googleCar.longTermMemory[key], oheTemp])
    
#     for trio in batch_vision_array_ball:
#         batch_vision_array[0] = np.append(batch_vision_array[0],trio[0]) #Vision values
#         batch_vision_array[1] = np.append(batch_vision_array[1],trio[1]) #score values
#         batch_vision_array[2] = np.append(batch_vision_array[2],trio[2]) #OHE
    
#     batch_vision_array[0] = batch_vision_array[0].reshape(int(len(batch_vision_array[0])/(numOfInputs*2)),(numOfInputs*2))
#     batch_vision_array[1] = scaler.fit_transform(batch_vision_array[1].reshape(int(len(batch_vision_array[1])/numOfOutputs),numOfOutputs))
#     batch_vision_array[2] = batch_vision_array[2].reshape(int(len(batch_vision_array[2])/numOfOutputs),numOfOutputs)
    
#     googleCar.model.fit([batch_vision_array[0], batch_vision_array[2]], batch_vision_array[1], batch_size=64, epochs=numOfEpochs, verbose = verbose, callbacks = callBackser, validation_split = val_split) 

# def fit_batch2_all(bench_mark, callBackser = None):
    
#     count = 0
#     # callbacksersList = ["val_loss", "val_acc" "checkpointer_earlystoper_car_score", None]
        
#     if False: #counter % 200 == 0:
#         verbose = 1
#     else:
#         verbose = 0
    
#     if callBackser == None:
#         numOfEpochs = 5
#         val_split = 0
#     else:
#         numOfEpochs = 9999999
#         if callBackser == "val_loss" or callBackser == "val_acc":
#             #checkpointer_val_loss = ModelCheckpoint("temp.hdf5", monitor="val_loss", mode="auto", save_best_only = True, verbose = verbose)
#             callBackser = [EarlyStopping(monitor = callBackser, patience = 10, verbose = verbose, restore_best_weights = True)]
#             val_split = 0.33
#         elif callBackser == "checkpointer_earlystoper_car_score":
#             callBackser = [EarlyStoppingAtBestCarScore(patience = 15, best_car_score = bench_mark, init_weights = googleCar.model.get_weights())]
#             val_split = 0
#         else:
#             raise NotImplementedError
            
#     batch_vision_array = [np.array([]),np.array([]),np.array([])]
#     batch_vision_array_ball = []

#     for key in (googleCar.longTermMemory):
#         if googleCar.longTermMemory[key].all != 0: 
#             count+=1
#             temp=key.split(",")
#             #oheTemp = np.array([1,1,1,1,1])
# #             for tempVal in googleCar.longTermMemory[key]:
# #                 if tempVal == 0:
# #                     oheTemp = np.append(oheTemp, 0)
# #                 else:
# #                     oheTemp = np.append(oheTemp, 1)

#             batch_vision_array_ball.append([googleCar.longTermMemoryVisionArray[key], googleCar.longTermMemory[key], np.array([1,1,1,1,1])])
    
#     for trio in batch_vision_array_ball:
#         batch_vision_array[0] = np.append(batch_vision_array[0],trio[0]) #Vision values
#         batch_vision_array[1] = np.append(batch_vision_array[1],trio[1]) #score values
#         batch_vision_array[2] = np.append(batch_vision_array[2],trio[2]) #OHE
        
#     try:
#         batch_vision_array[0] = batch_vision_array[0].reshape(int(len(batch_vision_array[0])/(numOfInputs*2)),(numOfInputs*2))
#         batch_vision_array[1] = scaler.fit_transform(batch_vision_array[1].reshape(int(len(batch_vision_array[1])/numOfOutputs),numOfOutputs))
#         batch_vision_array[2] = batch_vision_array[2].reshape(int(len(batch_vision_array[2])/numOfOutputs),numOfOutputs)
#         googleCar.model.fit([batch_vision_array[0], batch_vision_array[2]], batch_vision_array[1], batch_size=64, epochs=numOfEpochs, verbose = verbose, callbacks = callBackser, validation_split = val_split) 
    
#     except ValueError:
#         if verbose >= 1:
#             print("No valid inputs were found...")

In [9]:
#Uncomment below code to launch configurations
#I used scp to sync states across two machines. 
#You can remove it references the os.system to remove this aspect
# model_number_id = 0
# max_time = 60*60*8 #8 hours
# num_of_attempts = 3
# your_address = "/home/keras/google_car/"
# foreign_address = "keras@10.0.0.233:/home/keras/googlecar/"
# for numOfHiddenUnitsTemp in numberOfHiddenUnitsList:
#     for discountTemp in discountList:
#         for stepTemp in stepList:
#             for learning_rateTemp in learning_rateList:
#                 for numOfOutputsTemp in numOfOutputsList:
#                     for numOfInputsTemp in numOfInputsList:
#                         for death_penaltyTemp in death_penaltyList:
#                             for loopLengthTemp in loopLengthList:
#                                 for stochasticismFloorTemp, stochasticismRoofTemp in stochasticismRoofFloorList:
#                                     for callbacksersTemp in callbacksersList:
#                                         for anyOrAllTemp in anyOrAllList:
#                                             if not os.path.isfile(f"{your_address}{model_number_id}.conf"):
#                                                 os.system(f"scp -i /home/keras/.ssh/keras_keys {foreign_address}{model_number_id}.conf {your_address}")
#                                             if not os.path.isfile(f"{your_address}{model_number_id}.conf"):
#                                                 configurationBall = []
#                                                 configurationBall.append("CONFIGURATION")
#                                                 configurationBall.append(f"Number of hidden units: {numOfHiddenUnitsTemp}")
#                                                 configurationBall.append(f"Step: {stepTemp}")
#                                                 configurationBall.append(f"Discount: {discountTemp}")
#                                                 configurationBall.append(f"Learning rate: {learning_rateTemp}")
#                                                 configurationBall.append(f"Number Of Outputs: {numOfOutputsTemp}")
#                                                 configurationBall.append(f"Number Of Inputs: {numOfInputsTemp}")
#                                                 configurationBall.append(f"Death penalty: {death_penaltyTemp}")
#                                                 configurationBall.append(f"Model Callback: {callbacksersTemp}")
#                                                 configurationBall.append(f"Number of attempts inbetween training: {loopLengthTemp}")
#                                                 configurationBall.append(f"Stochasticism min, Stochasticism max: {stochasticismFloorTemp}, {stochasticismRoofTemp}")
#                                                 configurationBall.append(f"Which data to use: {anyOrAllTemp}")
#                                                 print("\n".join(configurationBall))
#                                                 open(str(model_number_id)+".conf", "w+").write("\n".join(configurationBall) + "\n")
#                                                 loopLength = loopLengthTemp
#                                                 numOfOutputs = numOfOutputsTemp
#                                                 numOfInputs = numOfInputsTemp
#                                                 death_penalty = death_penaltyTemp
#                                                 LEARNING_RATE = learning_rateTemp
#                                                 start = time.time()
#                                                 results = collect_attempts(num_of_attempts, max_time, str(model_number_id))
#                                                 print("results: ", results)
#                                                 print(f"max_time: {max_time}")
#                                                 open(str(model_number_id)+".conf", "a+").write(json.dumps(results) + "\n")
#                                             model_number_id+=1
                                            